This notebook explains how to run a simple 2-dimension example of [`Diva`](https://github.com/gher-ulg/DIVA).    
The input files and the figures are created within the notebook.    
The reading/writing of input files is done through specific classes defined in [pydiva2d](../pydiva/pydiva2d.py).

In [ ]:
import os
import logging
import shutil
import subprocess
import numpy as np
from pydiva import pydiva2d

# Setup 

## Logging configuration

The *logging* is already configured in `pydiva2d`.<br>
Replace 'DEBUG' by 'INFO', 'WARNING' or 'ERROR'.

In [ ]:
logger = logging.getLogger('pydiva2d')
logger.setLevel(logging.DEBUG)

## Diva configuration

Set the path to the `Diva` installation you want to work with.

In [ ]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"

# Prepare the input files
## Directories
Create variables storing the `Diva` directories.

In [ ]:
DivaDirs = pydiva2d.DivaDirectories(divadir)

Create variables storing the main `Diva` input and output files.

In [ ]:
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

## Contour

Create a sequence of points representing the contour (no crossing, no duplicate points, etc).     
It is not necessary to close the contour (i.e., setting the last point equal to the first point).    
Each coordinate (lon, lat) is defined as a [ndarray](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) or a list of lists (one per sub-contour).

In [ ]:
lon = np.array([[0., 2., 2., 0.], [2.5, 6., 6., 2.5]])
lat = np.array([[0., 0., 4., 4.], [0., 0., 3., 3.]]) 

Create a **contour** object using the coordinates: 

In [ ]:
contour2D = pydiva2d.Diva2DContours(lon, lat)

and write its content to the corresponding file defined in the **DivaFile** object:

In [ ]:
contour2D.write_to(DivaFiles.contour)

## Data

Create arrays of coordinates (*x*, *y*), values (*z*) and optionnaly weights (*w*) representing the data points.     
If the weights are not specified, it is assumed that all the data points have an identical weight (equal to 1).

In [ ]:
x = np.array((1.2, 1.75, 3., 5.5))  
y = np.array((2.1, 3.5, 2., 0.2))
z = np.array((1., -2., 4, -1))
w = np.array((1., 1., 0.5, 0.5))

Similarly to the contour, we create a **data** object:

In [ ]:
data2D = pydiva2d.Diva2DData(x, y, z, w)

and we write the values to the specified file:

In [ ]:
data2D.write_to(DivaFiles.data)

## Parameters

We need to set the parameters defining the domain and the analysis parameters.    
Check the [`Diva` documentation](http://github.com/gher-ulg/) to have a complete description of each parameter.

First we set the output grid (limits and spatial resolution).

In [ ]:
xmin, ymin, nx, ny, dx, dy = 0.0, 0.0, 61, 81, 0.1, 0.05

Then the parameters specific to the analysis:

In [ ]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -999.
iCoordChange = 0
iSpec = 11
iReg = 0

We create a **parameter** object that stores all the parameter values:

In [ ]:
parameters2D = pydiva2d.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

and we write them into a file:

In [ ]:
parameters2D.write_to(DivaFiles.parameter)

## List of extra points

In the optional input file *valatxy.coord*, one can specify additional locations where the interpolation has to be performed.<br>
Here we ask for the analysis at the location of the first two data points, plus at an additional location.

In [ ]:
xcoord = np.array((1.2, 1.75, 5.9))  
ycoord = np.array((2.1, 3.5, 3.9))

We create the object `valatxy` and write the coordinates to a file:

In [ ]:
valatxy = pydiva2d.Diva2DValatxy(xcoord, ycoord)
valatxy.write_to(DivaFiles.valatxy)

# Run Diva
Now that the input files have been created we can perform a Diva analysis.
## Mesh generation
We use the `make` method of the class `Diva2DMesh`. If we don't specify input files, the method takes those located in the *./input* directory by default.

In [ ]:
mesh2d = pydiva2d.Diva2DMesh().make(divadir)

## Run the diva calculation

In [ ]:
analysis2d = pydiva2d.Diva2DResults().make(divadir)

# Time for some plots

We import a few more modules to plot the input files content and the results.<br>
The last line asks for the plot to be done within the notebook, not as external window.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.size': 14, 'figure.dpi': 300})

## Input files

Let's represent the observations, the grid and the contours.<br>
By default, each contour is represented in a different color.

In [ ]:
fig = plt.figure()
contour2D.add_to_plot(linewidth=2)
data2D.add_to_plot(s=10)
parameters2D.plot_outputgrid()
plt.show()
plt.close()

## Results 

The results are available in the `analysis2d` object.

### Analysis

The field is then added to the plot. In addition we can include the locations of the observations.    
Note that, due to their lower weights, the 2 data points in the right-side domain seems to influence less the analysis.

In [ ]:
fig = plt.figure()
field2plot = analysis2d.add_to_plot(field='analysis')
data2D.add_to_plot(s=10)
plt.colorbar(field2plot)
plt.title('Analysed field and observations')
plt.show()
plt.close()

### Error field

The error field is represented using a similar command.<br>
Also note the larger error values in the right-hand side, due to the lower weights of the data points in that part of the domain.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
errorfield = analysis2d.add_to_plot(field='error', cmap=plt.cm.hot_r)
data2D.add_positions_to_plot(s=3, color='k')
plt.colorbar(errorfield)
plt.title('Relative error field and data locations')
plt.show()
plt.close()

## Finite-element mesh

It is also interesting to display the finite-element mesh that covers our domain of interest.    

The plot is prepared with the `add_to_plot` method:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
mesh2d.add_to_plot(lw=0.1)
contour2D.add_to_plot()
plt.title('Finite-element mesh and contours')
plt.show()
plt.close()